In [193]:
%matplotlib inline

from dask.distributed import Client
import xarray as xr
import os
import numpy as np
import dask
import glob
import pandas as pd
import datetime as dt
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

In [194]:
latitude = -27.45
longitude = 360 - 59.05

def my_fun(a):
    return a.integrate('step', '6h')
    
def seleccionar_punto(ds, lat=latitude, lon=longitude):
    ds = ds.sel(latitude=lat, longitude=lon, method='nearest')
    ds = ds.isel(step=range(32 * 4))
    ds['step'] = ds.valid_time.values - np.timedelta64(3, 'h')
    ds = ds.groupby("step.dayofyear").map(my_fun)
    ds = ds * 6 * 60 * 60 * 1e-6  #Mj/m2
    ds = ds.rename({'dayofyear': 'step'})
    #ds = ds.isel(step= (ds['step'] - ds['time.dayofyear']) > 0)
    #ds['step'] = (ds['step'] - ds['time.dayofyear'] - 1).astype('timedelta64[D]')
    ds['step'] = np.arange(len(ds['step'])).astype('timedelta64[D]')
    #ds['step'] = ds['step'].dt.floor('D')
    return ds
def modify(ds, lat=latitude, lon=longitude):
    try:
        ds = ds.sel(latitude=lat, longitude=lon, method='nearest')
        ds = ds.isel(step=range(32 * 4))
        ds['step'] = ds.valid_time.values - np.timedelta64(3, 'h')
        ds = ds.groupby("step.dayofyear").map(my_fun)
        ds = ds * 6 * 60 * 60 * 1e-6  #Mj/m2
        ds = ds.rename({'dayofyear': 'step'})
        #ds = ds.isel(step= (ds['step'] - ds['time.dayofyear']) > 0)
        #ds['step'] = (ds['step'] - ds['time.dayofyear'] - 1).astype('timedelta64[D]')
        ds['step'] = np.arange(len(ds['step'])).astype('timedelta64[D]')
        #ds['step'] = ds['step'].dt.floor('D')
        return ds
    except:
        pass
def create_summary_file(var, fval, fmat, leadtime):
    """
    Function to create a summary file to save the results
    """
    columnas = ['fecha']
    columnas.extend([var + '_' + hr for hr in ['00', '06', '12', '18']])
    df = pd.DataFrame(columns=columnas)
    df['fecha'] = [i + np.timedelta64(leadtime, 'D') for i in fval]
    df[columnas[1::]] = fmat
    return df


In [195]:
client = Client()
client

/home/osman/anaconda3/envs/py37/lib/python3.7/site-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46005 instead
  expected, actual


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:46005/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:46005/status,Workers: 8
Total threads: 40,Total memory: 62.83 GiB
Comm: tcp://127.0.0.1:34807,Workers: 8
Dashboard: http://127.0.0.1:46005/status,Total threads: 40
Started: Just now,Total memory: 62.83 GiB
Comm: tcp://127.0.0.1:43275,Total threads: 5
Dashboard: http://127.0.0.1:37547/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:37645,


In [200]:
var_name = 'radsup'
variable = 'dswsfc'
for i in np.arange(1999, 2011):
    for j in range(1, 13):
        if not(os.path.isfile(var_name + '_' + str(i) + '_' + '{:02d}'.format(j) + '.nc')):
            try: 
                ds = xr.open_mfdataset('/pikachu/datos2/CFSReana/' + variable + '/' + str(i) + '/' + variable + '_f.01.' + str(i) + '{:02d}'.format(j) + '*.grb2',
                                       engine='cfgrib', combine='nested', concat_dim='time', parallel=True, data_vars='minimal', coords='minimal', compat='override',
                                       preprocess=seleccionar_punto, backend_kwargs={'filter_by_keys':{'name':'Downward short-wave radiation flux'}})
                ds = ds.chunk({'time':len(ds.time), 'step':len(ds.step)})
                ds = ds.load()
                ds.to_netcdf(var_name + '_' + str(i) + '_' + '{:02d}'.format(j) + '.nc')
            except:
                # this is basically what open_mfdataset does
                open_kwargs = dict(decode_cf=True, engine='cfgrib')
                file_names = sorted(glob.glob('/pikachu/datos2/CFSReana/' + variable + '/' + str(i) + '/' + variable + '_f.01.' + str(i) + '{:02d}'.format(j) + '*.grb2'))
                open_tasks = [dask.delayed(xr.open_dataset)(f, **open_kwargs) for f in file_names]
                tasks = [dask.delayed(modify)(task) for task in open_tasks]
                datasets = dask.compute(tasks)  # get a list of xarray.Datasets
                combined = xr.combine_nested([f for f in datasets[0] if f ], concat_dim='time')
                combined.to_netcdf(var_name + '_' + str(i) + '_' + '{:02d}'.format(j) + '.nc')
ds = xr.open_mfdataset('radsup_*.nc', combine='nested', concat_dim='time', parallel=True, data_vars='minimal', coords='minimal', compat='override')
ds = ds.isel(step=range(31))
ds = ds.chunk({'time':int(len(ds.time)/2), 'step':len(ds.step)})
ds = ds.load()
ds = ds.isel(time= np.unique(ds.time, return_index=True)[1])
tiempos = np.arange(np.datetime64('1999-01-01').astype('datetime64[ns]'), np.datetime64('2011-01-01').astype('datetime64[ns]'), np.timedelta64(6, 'h')).astype('datetime64[ns]')
ds = ds.reindex(time=tiempos)
ds = ds.dswrf.to_numpy()
ds = np.reshape(ds, [int(len(tiempos)/4), 4, 31])
ds [:, 2: 4, 0] = np.nan
tiempos = np.arange(np.datetime64('1999-01-01').astype('datetime64[ns]'), np.datetime64('2011-01-01').astype('datetime64[ns]'), np.timedelta64(1, 'D')).astype('datetime64[ns]')
for i in range(31):
    df = create_summary_file(variable, tiempos, ds[:, :, i], i)
    #print(df)
    sel_col = list(df)[1::]
    df[sel_col] = df[sel_col].apply(pd.to_numeric, errors='ignore')
    n_file = 'data_final_' + var_name + '_' + "{:02d}".format(i) + '.txt' 
    df.to_csv(n_file, sep=';', float_format='%.2f', decimal=',',
                        date_format='%Y-%m-%d')

